In [1]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading Input Data
data = pd.read_csv('InputFile.csv')

In [3]:
data.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,negative
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,negative


In [4]:
data.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'Class'],
      dtype='object')

In [5]:
# Dropping unnecessary columns or redundant columns
data = data.drop(['TSH_measured','T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured','TBG','TSH'], axis=1)

In [7]:
#Checking for invalid data in the columns
for column in data.columns:
    count = data[column][data[column]=='?'].count()
    print(column,count)

age 1
sex 150
on_thyroxine 0
query_on_thyroxine 0
on_antithyroid_medication 0
sick 0
pregnant 0
thyroid_surgery 0
I131_treatment 0
query_hypothyroid 0
query_hyperthyroid 0
lithium 0
goitre 0
tumor 0
hypopituitary 0
psych 0
T3 769
TT4 231
T4U 387
FTI 385
referral_source 0
Class 0


In [8]:
#Replacing the invalid data with Nan values
for column in data.columns:
    count = data[column][data[column]=='?'].count()
    if count!=0:
        data[column] = data[column].replace('?',np.nan)

In [11]:
data['sex'].unique()

array(['F', 'M', nan], dtype=object)

In [12]:
#Mapping categorical values to numerical values
data['sex'] = data['sex'].map({'F':0,'M':1})

In [14]:
for column in data.columns:
    if len(data[column].unique())==2:
        data[column] = data[column].map({'f':0,'t':1})

In [16]:
data.referral_source.unique()

array(['SVHC', 'other', 'SVI', 'STMW', 'SVHD'], dtype=object)

In [17]:
#Converting Categorical values to numerical values
data = pd.get_dummies(data,columns=['referral_source'])

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
#Encoding Class variable using LabelEncoder
encode = LabelEncoder().fit(data['Class'])
data['Class'] = encode.transform(data['Class'])

In [23]:
import pickle

In [24]:
# saving the encoded file so that we can use it later to decode it
with open('enc.pickle','wb') as file:
    pickle.dump(encode,file)

In [25]:
# Dividing dataset into independent set and dependent set
X = data.drop(['Class'],axis=1)
y = data['Class']

In [27]:
#Checking for null values across all columns
X.isna().sum()

age                            1
sex                          150
on_thyroxine                   0
query_on_thyroxine             0
on_antithyroid_medication      0
sick                           0
pregnant                       0
thyroid_surgery                0
I131_treatment                 0
query_hypothyroid              0
query_hyperthyroid             0
lithium                        0
goitre                         0
tumor                          0
hypopituitary                  0
psych                          0
T3                           769
TT4                          231
T4U                          387
FTI                          385
referral_source_STMW           0
referral_source_SVHC           0
referral_source_SVHD           0
referral_source_SVI            0
referral_source_other          0
dtype: int64

In [28]:
from sklearn.impute import KNNImputer

In [29]:
# Replace null values with KNN imputation
imputer = KNNImputer(n_neighbors=3,weights='uniform',missing_values=np.nan)
X_impute = imputer.fit_transform(X)
X = pd.DataFrame(data=np.round(X_impute),columns=X.columns)

In [33]:
X.isna().sum()

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
T3                           0
TT4                          0
T4U                          0
FTI                          0
referral_source_STMW         0
referral_source_SVHC         0
referral_source_SVHD         0
referral_source_SVI          0
referral_source_other        0
dtype: int64

In [34]:
y.value_counts()

1    3481
0     194
2      95
3       2
Name: Class, dtype: int64

In [36]:
# Converting imbalanced dataset into balanced dataset using RandomOverSampler
from imblearn.over_sampling import RandomOverSampler
random = RandomOverSampler()
x_sampled,y_sampled = random.fit_sample(X,y)

In [39]:
# Finding the 'k' value using elbow method
from sklearn.cluster import KMeans
from kneed import KneeLocator

wcss = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=10)
    kmeans.fit(x_sampled)
    wcss.append(kmeans.inertia_)


plt.plot(range(1,11),wcss)
plt.title('Elbow Method')
plt.xlabel('No of Clusters')
plt.ylabel('WCSS')
plt.savefig('elbow_method.png')


In [ ]:
#Applying kmeans algorithm with all the clusters
clusters = KneeLocator(range(1,11),wcss,curve='convex',direction='decreasing')
kmeans = KMeans(n_clusters=clusters.knee, init='k-means++', random_state=10)
y_kmeans = kmeans.fit_predict(x_sampled)

In [46]:
# Save the models
import os
import shutil

model_dir = 'models/'
path = os.path.join(model_dir, 'K Means')

if os.path.isdir(path):
    shutil.rmtree(model_dir)
    os.makedirs(path)
else:
    os.makedirs(path)
    
with open(path+'/'+'K Means'+'.sav', 'wb') as f:
    pickle.dump(kmeans,f)

In [50]:
x_sampled['Cluster'] = y_kmeans
x_sampled['Labels'] = y_sampled

list_of_clusters = x_sampled['Cluster'].unique()


In [95]:
#Function to save the model
def savemodel(model,model_name):
    model_dir = 'models/'
    path = os.path.join(model_dir, model_name)
    
    if os.path.isdir(path):
        shutil.rmtree(model_dir)
        os.makedirs(path)
    else:
        os.makedirs(path)
    
    with open(path+'/'+model_name+'.sav','wb') as f:
        pickle.dump(model,f)

In [96]:
#Importing libraries for Modelling and metrics calculation.
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score

In [98]:
# Training the data with KNN Classifier and Random Forest Classifier. 
# Finally choose the best model betweent the two by comparing the accuracy score or roc_auc score.
for i in list_of_clusters:
    cluster_data = x_sampled[x_sampled['Cluster']==i]
    cluster_features = cluster_data.drop(['Labels','Cluster'],axis=1)
    cluster_label = cluster_data['Labels']
    
    #Dividing the dataset into train and test.
    x_train,x_test,y_train,y_test = train_test_split(cluster_features,cluster_label,test_size=1/3,random_state=7)
    
    #Finding best params for Knn by doing hyperparameter tuning.
    knn = KNeighborsClassifier()
    param_grid_knn = {
            'algorithm':['ball_tree','kd_tree','brute'],
            'leaf_size':[10,15,20,25,30,35],
            'n_neighbors':[4,5,6,7,8,10],
            'p':[1,2]
        }
    grid_knn = GridSearchCV(knn,param_grid_knn,verbose=3,cv=5)
    grid_knn.fit(x_train,y_train)
    
    #Choosing the best parameters for Knn algo.
    algorithm = grid_knn.best_params_['algorithm']
    leaf_size = grid_knn.best_params_['leaf_size']
    n_neighbors = grid_knn.best_params_['n_neighbors']
    p = grid_knn.best_params_['p']
    
    #Applying Knn classifier with the best parameters obtained
    knn_new = KNeighborsClassifier(algorithm=algorithm,leaf_size=leaf_size,n_neighbors=n_neighbors,p=p,n_jobs=-1)
    knn_new = knn_new.fit(x_train,y_train)
    knn_predict = knn_new.predict_proba(x_test)
    
    #Checking the performance using accuracy/roc_auc score.
    if len(y_test.unique()) == 1:
        knn_score = accuracy_score(y_test,knn_predict)
    else:
        knn_score = roc_auc_score(y_test,knn_predict,multi_class='ovr')
    
    
    #Finding best params for Random Forest by using hyperparameter tuning.
    clf = RandomForestClassifier()
    
    param_grid_rf = {
    'n_estimators':[10,50,100,130],
    'criterion':['gini','entropy'],
    'max_depth':range(2,4,1),
    'max_features':['auto','log2']
    }
    
    grid_rf = GridSearchCV(estimator=clf,param_grid=param_grid_rf,cv=5,verbose=3)
    grid_rf.fit(x_train,y_train)
    
    #Obtaining the best parameters.
    criterion = grid_rf.best_params_['criterion']
    max_depth = grid_rf.best_params_['max_depth']
    max_features = grid_rf.best_params_['max_features']
    n_estimators = grid_rf.best_params_['n_estimators']
    
    # Applying Random Forest Classifier with the best parameters obtained from cross validation
    clf_rf = RandomForestClassifier(n_estimators=n_estimators,criterion=criterion,max_depth=max_depth,max_features=max_features)
    clf_rf = clf_rf.fit(x_train,y_train)
    predict_rf = clf_rf.predict_proba(x_test)
    
    # Checking the performance using accuracy score or roc_auc score.
    if len(y_test.unique())==1:
        rf_score = accuracy_score(y_test,predict_rf)
    else:
        rf_score = roc_auc_score(y_test,predict_rf,multi_class='ovr')
    
    #Finding the best model by comparing the performance scores of the two models.
    if (rf_score < knn_score):
        print('KNN', knn_new)
        best_model_name='KNN'
        best_model = knn_new
    else:
        print('Random Forest:',clf_rf)
        best_model_name='Random Forest'
        best_model=clf_rf
    
    # Saving the best Model.
    save_model = savemodel(best_model,best_model_name+str(i))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1, score=0.892, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1, score=0.885, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1, score=0.903, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1, score=0.921, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1 ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1, score=0.882, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2, score=0.897, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2, score=0.897, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2, score=0.907, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2, score=0.927, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2, score=0.900, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=5, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=10, n_nei

[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2, score=0.806, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2 ..........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2, score=0.816, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2 ..........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2, score=0.874, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2 ..........
[CV]  algorithm=ball_tree, leaf_size=10, n_neighbors=10, p=2, score=0.816, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=4, p=1, score=0.891, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=4, p=1, score=0.885, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=4, p=1 ...........
[CV]  algorithm=ball_tree, leaf_size=15, n

[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=8, p=2, score=0.892, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=8, p=2 ...........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=8, p=2, score=0.840, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1 ..........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1, score=0.819, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1 ..........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1, score=0.807, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1 ..........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1, score=0.818, total=   0.0s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1 ..........
[CV]  algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1, score=0.857, total=   0.1s
[CV] algorithm=ball_tree, leaf_size=15, n_neighbors=10, p=1 ..........
[CV]  algorithm=ball_tree, leaf_size=15, n

...

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=10 .
[CV]  criterion=gini, max_depth=2, max_features=auto, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=10 .
[CV]  criterion=gini, max_depth=2, max_features=auto, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=10 .
[CV]  criterion=gini, max_depth=2, max_features=auto, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=10 .
[CV]  criterion=gini, max_depth=2, max_features=auto, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=10 .
[CV]  criterion=gini, max_depth=2, max_features=auto, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=gini, max_depth=2, max_features=auto, n_estimators=50 .
[CV]  criterion=gini, 

[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=50, score=0.998, total=   0.1s
[CV] criterion=gini, max_depth=3, max_features=auto, n_estimators=100 
[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=100, score=0.998, total=   0.1s
[CV] criterion=gini, max_depth=3, max_features=auto, n_estimators=100 
[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=100, score=0.998, total=   0.1s
[CV] criterion=gini, max_depth=3, max_features=auto, n_estimators=100 
[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=100, score=0.998, total=   0.1s
[CV] criterion=gini, max_depth=3, max_features=auto, n_estimators=100 
[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=100, score=0.998, total=   0.2s
[CV] criterion=gini, max_depth=3, max_features=auto, n_estimators=100 
[CV]  criterion=gini, max_depth=3, max_features=auto, n_estimators=100, score=0.998, total=   0.1s
[CV] criterion=gini, max_depth=3, max_features=auto,

[CV]  criterion=entropy, max_depth=2, max_features=auto, n_estimators=130, score=0.998, total=   0.2s
[CV] criterion=entropy, max_depth=2, max_features=auto, n_estimators=130 
[CV]  criterion=entropy, max_depth=2, max_features=auto, n_estimators=130, score=0.998, total=   0.2s
[CV] criterion=entropy, max_depth=2, max_features=auto, n_estimators=130 
[CV]  criterion=entropy, max_depth=2, max_features=auto, n_estimators=130, score=0.998, total=   0.2s
[CV] criterion=entropy, max_depth=2, max_features=log2, n_estimators=10 
[CV]  criterion=entropy, max_depth=2, max_features=log2, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=entropy, max_depth=2, max_features=log2, n_estimators=10 
[CV]  criterion=entropy, max_depth=2, max_features=log2, n_estimators=10, score=0.981, total=   0.0s
[CV] criterion=entropy, max_depth=2, max_features=log2, n_estimators=10 
[CV]  criterion=entropy, max_depth=2, max_features=log2, n_estimators=10, score=0.998, total=   0.0s
[CV] criterion=entropy, 

[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=50, score=0.998, total=   0.1s
[CV] criterion=entropy, max_depth=3, max_features=log2, n_estimators=50 
[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=50, score=0.998, total=   0.1s
[CV] criterion=entropy, max_depth=3, max_features=log2, n_estimators=50 
[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=50, score=0.998, total=   0.1s
[CV] criterion=entropy, max_depth=3, max_features=log2, n_estimators=50 
[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=50, score=0.998, total=   0.1s
[CV] criterion=entropy, max_depth=3, max_features=log2, n_estimators=100 
[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=100, score=0.998, total=   0.1s
[CV] criterion=entropy, max_depth=3, max_features=log2, n_estimators=100 
[CV]  criterion=entropy, max_depth=3, max_features=log2, n_estimators=100, score=0.998, total=   0.2s
[CV] criterion=entropy, m

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   17.7s finished


In [ ]:
#Prediction 

In [99]:
#Read the file for prediction
data_p = pd.read_csv('PredictFile.csv')

In [101]:
#Dropping unnecessary columns
data_p = data_p.drop(['TSH_measured','T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured','TBG','TSH'], axis=1)

In [105]:
#Checking for invalid values across all the columns
for column in data_p.columns:
    count = data_p[column][data_p[column]=='?'].count()
    print(column,count)

age 0
sex 13
on_thyroxine 0
query_on_thyroxine 0
on_antithyroid_medication 0
sick 0
pregnant 0
thyroid_surgery 0
I131_treatment 0
query_hypothyroid 0
query_hyperthyroid 0
lithium 0
goitre 0
tumor 0
hypopituitary 0
psych 0
T3 56
TT4 15
T4U 23
FTI 23
referral_source 0


In [106]:
#Replacing the invalid values with Nan 
for column in data_p.columns:
    count = data_p[column][data_p[column]=='?'].count()
    if count!=0:
        data_p[column] = data_p[column].replace('?',np.nan)

In [107]:
#Mapping categorical values to numerical values
data_p['sex'] = data_p['sex'].map({'F':0,'M':1})
cat_data = data_p.drop(['age','sex','T3','TT4','T4U','FTI'],axis=1)

for column in cat_data.columns:
    if data_p[column].nunique() == 1:
        if data_p[column].unique()[0] == 'f' or data_p[column].unique()[0]=='F':
            data_p[column] = data_p[column].map({data_p[column].unique()[0]:0})
        else:
            data_p[column] = data_p[column].map({data_p[column].unique()[0]:1})
    elif data_p[column].nunique() ==2:
        data_p[column] = data_p[column].map({'f':0,'t':1})

In [110]:
#COnverting categorical values to numerical values
data_p = pd.get_dummies(data_p, columns=['referral_source'])

In [111]:
#Checking for null values
null_count = data_p.isna().sum()

In [113]:
#Replacing null values with KNN Imputation
imputer = KNNImputer(n_neighbors=3,weights='uniform',missing_values=np.nan)   
impute_x = imputer.fit_transform(data_p)
data_p = pd.DataFrame(data=np.round(impute_x),columns=data_p.columns)

In [132]:
#Function to load the model
def load_model(filename):
    model_dir = 'models/'
    with open(model_dir+filename+'/'+filename+'.sav','rb') as f:
        return pickle.load(f)

In [133]:
kmeans_p = load_model('K Means')
clusters_p = kmeans_p.predict(data_p)
data_p['clusters'] = clusters_p
clusters_p = data_p['clusters'].unique()

In [138]:
with open('enc.pickle','rb') as file:
    encoder = pickle.load(file)

In [139]:
#Function to pick the model and find the model name.
def find_model_name(cluster_num):
    cluster_num = cluster_num
    folder_name = 'models/'
    list_of_models = []
    list_of_files = os.listdir(folder_name)
    
    for file in list_of_files:
        try:
            if file.index(str(cluster_num))!=-1:
                model_name=file
        except:
            continue
    
    model_name = model_name.split('.')[0]
    return model_name

In [142]:
import pandas as pd

In [146]:
#Making predictions on the inputted data. 
#Saving the predictions by copying it to the csv file.
result=[]
for i in clusters_p:
    cluster_data_p = data_p[data_p['clusters']==i]
    cluster_data_p = cluster_data_p.drop(['clusters'],axis=1)
    model_name= find_model_name(i)
    model = load_model(model_name)
    
    for val in (encoder.inverse_transform(model.predict(cluster_data_p))):
        result.append(val)

result = pd.DataFrame(result,columns=['Predictions'])
path='Predictions.csv'
result.to_csv('Predictions.csv',header=True)